# Explanation

In imitation learning environments, the model is indirectly learning to maximize reward in an environment by assuming that expert demonstrations are behaving with actions that maximize reward, and learning to replicate these behaviors. This is implicitly assuming that the expert has optimized their actions against the true environment reward function.

Inverse reinforcement learning makes the implication explicit by directly trying to learn the expert's reward function first, and then training a policy with this reward function. IRL iterates through loops of updating its reward function model and then updating a policy against it until the reward hits an optimal threshold.

Specifically, the model uses a pre-defined parametrization of the reward function that can be adjusted with a set of weights. This parametrization reflects what the model creators believe to be the set of viable factors that may be useful to consider in the reward function. 

At each step, the model then first tries to select the reward function where the expert performs most significantly better than the current policy, and then trains a policy with this reward function. This process helps the policy to converge toward the expert demonstration given a variety of different reward considerations.

The disadvantage of this approach is that the implicit parametrization of the reward function heavily limits what the model may learn is important in the environment. If important objectives are not included in the parametrization, the model may never learn to optimize toward it.

# Notes

> [Observing an expert demonstration] is useful in applications (such as the task of driving) where it may be difficult to write down an explicit reward function specifying exactly how different desiderata should be traded off.

Imitation learning is especially useful when the reward function is unclear or is a complex linear combination of many factors that are hard to explicitly quantify.

> The MDP formalism is useful for many problems because it is often easier to specify the reward function than to directly specify the value function (or optimal policy).

The whole reason we optimize against the reward function is that we believe we can construct the reward function much more easily than we can the policy.

> However, we believe that even the reward function is frequently difficult to specify manually.

In real environments, the reward function itself can be intractable.

> We believe that the difficulty of manually specifying a reward function represents a significant barrier to the broader applicability of reinforcement learning and optimal control algorithms.

> The entire field of reinforcement learning is founded on the presupposition that the reward function […] is the most succinct, robust, and transferable definition of the task.

This is the core assumption behind many reinforcement learning approaches. The reward function and the environment contain all the information necessary to complete the task.

IRL instead deals with scenarios where the reward function is intractable because it isn’t directly accessible, whereas the policy can be observed and imitated to simulate the reward function it’s optimize for.

> The problem of deriving a reward function from observed behavior is referred to as **inverse reinforcement learning.**

### Preliminaries

We define MDP\R to be an MDP with no explicit reward function of the form $(S, A, T, \gamma, D)$.

We assume there some vector of features $\phi: S \rightarrow [0, 1]^k$ being factored into a “true” reward function $R^*(s) = w^* \cdot \phi(s)$, with $w^*$ representing the relative weighting of importance of the different features in $\phi$.

This represents the assumption that the true reward function represents a linear combination of some of the features learnable from $s$.

With this reward function, we then have the expected value of a policy $\pi$ over the infinite trajectories with starting state $s_0$ sampled from $D$ as:

$$
\mathbb{E}_{s_0 \sim D}[V^\pi(s_0)] = \mathbb{E}[\sum_{t=0}^\infty \gamma^t R(s_t)|\pi]
$$

We can fill in our reward function to get the expected value determined by the features $\phi$ and their relative importances:

$$
\mathbb{E}_{s_0 \sim D}[V^\pi(s_0)] = w \cdot \mathbb{E}[\sum_{t=0}^\infty \gamma^t \phi(s_t)|\pi]
$$

Then we can define the **feature expectations** to be:

$$
\mu(\pi) = \mathbb{E}\left[  \sum_{t=0}^\infty \gamma^t \phi(s_t)|\pi \right] \in \mathbb{R}^k
$$

And the reward function can be redefined as $\mathbb{E}_{s_0 \sim D}[V^\pi(s_0)] = w \cdot \mu(\pi)$.

We also assume access to demonstrations by an expert policy $\pi_E$. Given a set of trajectories sampled from $\pi_E$ (expert demonstrations), we can approximate the experts feature expectations $\mu_E$:

$$
\hat{\mu}_E = \frac{1}{m} \sum_{i=1}^m \sum_{t=0}^\infty \gamma^t \phi(s_t^{(i)})
$$

Which we have access to because we can compute the features $\phi$ on the states $s_t$ visited by the expert in each trajectory.

This should approximately show us the features that the expert most values, giving us an approximation of $w$.

### Algorithm

Given an MDP\R, features $\phi$, and the expert feature expectations $\mu_E$ approximated from demonstrations, we have to find a policy that has similar performance to the expert on the unknown underlying reward function $R^* = w^{*T}\phi$.

Then we have to find a policy $\hat{\pi}$ such that $|| \mu(\hat{\pi}) - \mu_E ||_2 \leq \epsilon$, where $\epsilon$ specifies the error threshold between the feature expectations. This is the optimization because the actual reward and performance of the policy follow from the feature expectations.

To find this policy $\hat{\pi}$, we take the following steps:

1. Pick a random policy $\pi^{(0)}$ and compute $\mu^{(0)}$ for it
2. Select the value for $w$ that maximizes the difference between $w^T\mu_E$ and $w^T \mu^{(i)}$ of the best policy you have so far. This gives you access to the underlying reward function $R^*$ in which your policy would perform the worst compared with the expert policy. In this scenario, you get the error:

   $$
   t^{(i)} = \max_{w: ||w||_2 \leq 1} \min_{j \in \{ 0..(i-1) \}} w^T(\mu_E - \mu^{(j)})
   $$

3. If the worst case error $t^{(i)} \leq \epsilon$ then we’ve found an algorithm that’s good enough and can terminate.
4. Otherwise, use a traditional RL algorithm to get the best policy $\pi^{(i)}$ for the MDP with $R = (w^{(i)})^T \phi$. Here, we use the inferred reward function to train another RL algorithm.
5. Then we compute $\mu^{(i)}$ with this new policy, increment $i$ and repeat this process until we converge.

Step 2 is similar to finding the maximum margin hyperplane separating two sets of points from SVM.

The above method is called the **max-margin** method since it finds the reward function with the maximum margin of error between the policies in step 2.

There is also the **projection method** where we replace step 2 with a method that projects the last 2 policies and $\mu_E$ on a line and moves the policies toward $\mu_E$.

They also show that the algorithm will have performance that isn’t significantly worse than the experts so it will eventually terminate.

### Experiments

**1. Gridworld**

There are 64 macrocells, each of which have a feature $\phi_i(s)$ indicating if state $s$ is in macrocell $i$.

The true reward function specifies $R^* = (w^*)^T \phi$ with the weights $w^*$ generated randomly to give random and sparse rewards.

This is a bit of a contrived scenario because they’ve designed it so that the features perfectly match the reward function, which is only possible when you do in fact have the reward function already.

They also compare the algorithm to a bunch of random RL algorithms that aren’t actually good.

![Screenshot 2024-11-06 at 3.55.50 PM.png](../../../images/notes/Screenshot_2024-11-06_at_3.55.50_PM.png)

> Thus, by learning a compact representation of the reward function, our algorithm significantly outperforms the other methods.

> We also observe that when the algorithm is told in advance which features have non-zero weight in the true reward function, it is able to learn using fewer expert trajectories.

Even when they provide the reward function in the inverse reinforcement learning format (providing weights on features), their method works better.

**2. Car Driving Simulation**

> For our second experiment, we implemented a car driving simulation, and applied apprenticeship learning to try to learn different “driving styles.”

They create features indicating the cars current lane ( including off-road) and distance to other drivers.

They collected expert demonstrations of 5 different driving styles varying in which lane (in-lane vs. off-road right) and how nice to drive (nice vs. crash).

The IRL algorithm was able to learn all the different driving styles successfully.

This suggests that if the axes for features are chosen correctly, IRL allows the model to tune itself to learn the right reward function and then build a policy around it.

### Conclusions

> Our algorithm assumed the reward function is expressible as a linear function of known features. If the set of features is sufficiently rich, this assumption is fairly unrestrictive.

The selected feature set needs to be rich enough to express all practical reward functions. This also cannot learn reward functions that are non-linear combinations of the features.
